In [9]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers.flatten_observation import FlattenObservation
import sailboat_gym
import gymnasium as gym
import numpy as np

In [16]:
class Wrapper(gym.Wrapper):
    action_space = gym.spaces.Box(
        low=np.array([-1], dtype=np.float32),
        high=np.array([1], dtype=np.float32),
        shape=(1,),
        dtype=np.float32
    )

def create_env():
    env = gym.make('SailboatLSAEnv-v0')
    env = FlattenObservation(env)
    env = Wrapper(env)
    return env

env = DummyVecEnv([lambda: gym.make('SailboatLSAEnv-v0')])
model = PPO('MlpPolicy', env, seed=0)

[Parameter containing:
 tensor([[ 1.8142e-04,  5.0615e-04, -5.4414e-04,  2.0860e-03,  1.0398e-03,
          -1.2636e-03, -1.7357e-03,  9.5018e-05,  2.8166e-03,  1.8968e-03,
           2.3863e-03,  2.4235e-03,  5.1927e-04,  4.2267e-04,  1.1193e-03,
          -1.6527e-03,  7.1607e-04, -7.0886e-04, -1.1243e-03, -8.5450e-04,
           6.6423e-04,  4.3319e-05, -2.2462e-03, -1.1525e-03,  2.6175e-05,
           5.7202e-04, -6.1486e-04, -2.0330e-03,  6.0588e-04,  7.9633e-04,
           2.3311e-04,  2.5761e-05,  5.2500e-04, -1.5363e-03, -1.7530e-03,
          -3.8813e-04,  1.8074e-06, -1.4870e-03, -6.4326e-04,  2.6910e-03,
           8.7189e-04, -2.0200e-04,  1.6259e-04, -2.4959e-04,  7.4581e-06,
          -5.9592e-04, -1.4086e-03,  2.0858e-04,  1.6565e-04,  4.5475e-05,
           4.9452e-04,  2.8073e-04, -2.2126e-03, -1.2712e-03,  3.7405e-04,
           5.8539e-04,  1.5135e-03,  1.2774e-04,  4.6823e-04, -1.2023e-03,
           1.4492e-03,  2.4037e-03, -2.2898e-03,  1.6349e-03]],
        requi